In [2]:
import os
import pandas as pd
import geopandas as gpd

In [43]:
import re

In [3]:
import sys
sys.path.append('../')

In [4]:
report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.2\reports.gdb'

In [144]:
old_report_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.1\reports.gdb'

In [145]:
gpd.list_layers(old_report_path)

,name,geometry_type
0,activity_report20250814,Point
1,footprint2021,MultiPolygon
2,footprint2022,MultiPolygon
3,footprint2023,MultiPolygon
4,footprint2024,MultiPolygon


In [67]:
activity_out_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.2\activity_stats.xlsx'
footprint_out_path = r'D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0.2\footprint_stats.xlsx'

In [9]:
years = ['2021', '2022', '2023', '2024', '2025']

In [60]:
def get_reports(report_path, years):
    # list all available activity layers
    layers = gpd.list_layers(report_path).name
    # get activity layer name
    activity_layer_name = layers[layers.apply(lambda x: 'activity' in x)]
    if len(activity_layer_name) > 1:
        # if there are more than one activity reports generated from different dates available
        # find the latest one
        idx_max = activity_layer_name.apply(lambda x: re.findall(r'\d+', x)[0]).sort_values(ascending=False).index[0]
        activity_layer_name = activity_layer_name[idx_max]
    else:
        activity_layer_name = activity_layer_name[0]
    activity_gdf = gpd.read_file(report_path, driver='OpenFileGDB', layer=activity_layer_name)
    
    # limity activity report to current years
    activity_gdf = activity_gdf[activity_gdf['Year_txt'].isin(years)]
    
    # read all footprint by years
    footprint_list = []
    for y in years:
        footprint_list.append(gpd.read_file(report_path, driver='OpenFileGDB', layer='footprint'+ y))
    
    # add year_txt column
    for gdf, y in zip(footprint_list, years):
        gdf['Year_txt'] = y
    
    # concat to get final footprint
    footprint_gdf = pd.concat(footprint_list)
    
    return activity_gdf, footprint_gdf

In [61]:
activity_gdf, footprint_gdf = get_reports(report_path, years)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [146]:
years = ['2021', '2022', '2023', '2024']

In [147]:
old_activity_gdf, old_footprint_gdf = get_reports(old_report_path, years)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [32]:
def get_report_stats(df, col_name):
    # activity acres by input column name
    pivot_table = df.pivot_table(
        index=col_name,
        columns="Year_txt",
        values="ACTIVITY_QUANTITY",
        aggfunc="sum",
        fill_value=0
    )
    
    pivot_table.loc['TOTAL',:] = pivot_table.sum()
    return pivot_table

In [80]:
# add CORE_CRITERIA to append and report for database

# maybe use integer for CORE_CRITERIA

array(['VALID', 'FIXED'], dtype=object)

In [133]:
activity_cols = ['ADMINISTERING_ORG', 'PRIMARY_OWNERSHIP_GROUP', 'REGION', 'ACTIVITY_CAT', 'BROAD_VEGETATION_TYPE']
footprint_cols = ['PRIMARY_OWNERSHIP_GROUP', 'REGION', 'BROAD_VEGETATION_TYPE']

diff_cols = ['ACTIVITY_CAT', 'ADMINISTERING_ORG'] 

In [87]:
# remove time zone for excel output
activity_gdf['ACTIVITY_END'] = activity_gdf.ACTIVITY_END.dt.tz_localize(None)

In [154]:
def generate_excel(gdf, old_gdf, stats_cols, diff_cols, output_path, report_sheet_name):
    # init writer
    writer = pd.ExcelWriter(output_path, engine="xlsxwriter") 
    workbook = writer.book
    # create format oject that convert excel number format to comma seperated 2 decimal precision
    cellFormat = workbook.add_format({'num_format': '#,##.00'})
    # save original report to excel
    gdf.drop(columns=['geometry']).to_excel(writer, sheet_name=report_sheet_name) 
    
    
    for col in stats_cols:
        # save current summary statistic pivot table to sheet
        stats_df = get_report_stats(gdf, col)
        stats_df.to_excel(writer, sheet_name=col +'_stats')
        
        # format float to comma seperate with 2 decimal points
        worksheet = writer.sheets[col +'_stats']
        worksheet.set_column('A:{}'.format(chr(65+stats_df.shape[1])), stats_df.shape[0]+1, cellFormat)

        
    # create stat differences between versions
    for col in diff_cols:
        new_stats_df = get_report_stats(gdf, col)
        old_stats_df = get_report_stats(old_gdf, col)
        out_df = new_stats_df[old_stats_df.columns] - old_stats_df
        out_df.to_excel(writer, sheet_name=col +'_diff')
        
        # format float to comma seperate with 2 decimal points
        worksheet = writer.sheets[col +'_diff']
        worksheet.set_column('A:{}'.format(chr(65+out_df.shape[1])), out_df.shape[0]+1, cellFormat)
        
    writer.close()

In [152]:
generate_excel(activity_gdf, old_activity_gdf, activity_cols, diff_cols, activity_out_path, 'activity_report')

In [153]:
generate_excel(footprint_gdf, old_footprint_gdf, footprint_cols, footprint_cols, footprint_out_path, 'footprint_report')